In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline

Using TensorFlow backend.


In [2]:
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [3]:
from keras.datasets import mnist

In [4]:
from keras.datasets import cifar100

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

In [6]:
images=["Lenna.png"]

In [7]:
from keras.backend import clear_session

In [8]:
import numpy as np
import random
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf

In [9]:
y2=Image.open("Lenna.png").resize([28,28]).convert("L")

In [12]:
np.random.seed(seed=1)

os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
tf.random.set_seed(0)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
degree=[0,90,180,270]
values=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
convo=[6]
y2=Image.open("Lenna.png").resize([28,28]).convert("L")
for goal in images:
    #y2=Image.open(goal).resize([32,32])
    data2=mnist.load_data()[0][0]
    label2=mnist.load_data()[0][1]
    #label2=label2.reshape(len(label2))
    x_test=mnist.load_data()[1][0]
    y_test=mnist.load_data()[1][1]
    #y_test=y_test.reshape(len(y_test))
    #data3=np.zeros([len(data2),32,32,3])
    #for n in range(len(data2)):
        #data3[n]=Image.fromarray(data2[n].astype("uint8"))
    #data2=data3
    
    outputs=[]
    outputs4=[]
    outputs5=[]
    for c in range(len(convo)):
        outputs2=[]
        outputs3=[]
        for e in range(5):
            ensemble=[]
            for rot in degree:
                error_record=[]
                clear_session()
                unseen_score=np.zeros([10,9])
                AUC_score=np.zeros(10)
                for i in range(10):
                    x_train=data2[label2==i]
                    y_train=np.zeros([len(x_train),28,28,1])
                    for n in range(len(x_train)):
                        y_train[n]=np.array(y2.rotate(rot)).reshape(28,28,1)
                    #x_train=x_train/255
                    y_train=y_train
                    #x_test=data2[label2!=i]
                    #x_test=x_test/255
                    x_test=x_test.reshape(len(x_test),28,28,1)
                    y_label=np.zeros(len(y_test))
                    y_label[y_test==i]=1
                    #(x_train, train), (x_test, y_test) = mnist.load_data()
                    image_height, image_width = 28, 28
                    # バックエンドに依存したチャネルの位置を調整する
                    if K.image_data_format() == 'channels_last':
                        x_train = x_train.reshape(x_train.shape[0],
                                                  image_height, image_width,1)
                        #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                        y_train = y_train.reshape(y_train.shape[0],
                                                  28, 28,1)
                        input_shape = (image_height, image_width,1)
                    else:
                        x_train = x_train.reshape(x_train.shape[0],
                                                  1, image_height, image_width)
                        x_test = x_test.reshape(x_test.shape[0],1, image_height, image_width)
                        input_shape = (1, image_height, image_width)

                    # Min-Max Normalization (0. ~ 1. の範囲に値を収める)
                    x_train = x_train.astype('float32')
                    y_train = y_train.astype('float32')
                    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=e)
                    x_valid, x_valid2, y_valid, y_valid2 = train_test_split(x_valid, y_valid, test_size=0.3, random_state=e)
                    #x_test = x_test.astype('float32')
                    #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                    #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())
                    # 畳み込み演算を用いた AutoEncoder のネットワーク (Sequential API)
                    model = models.Sequential()
                    # 28 x 28 x 1
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                    for n in range(convo[c]):
                        model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                    model.add(layers.Conv2D(1, kernel_size=(3, 3),activation='relu', padding='same'))
                    # 28 x 28 x 1
                    model.compile(optimizer='adam',
                                  loss='mean_absolute_error')
                    # モデルの構造を確認する
                    fit_callbacks = [
                        callbacks.EarlyStopping(monitor='val_loss',
                                                patience=5,
                                                mode='min')
                    ]
                    # モデルを学習させる
                    model.fit(x_train, y_train,
                              epochs=100,
                              batch_size=16,
                              shuffle=True,
                              validation_data=(x_valid, y_valid),callbacks=fit_callbacks)
                    result=model.predict(x_valid2)
                    result2=model.predict(x_test)
                    y_train=y_train.reshape(y_train.shape[0],784,1)
                    result=result.reshape(result.shape[0],784,1)
                    result2=result2.reshape(result2.shape[0],784,1)
                    loss=np.zeros(len(result))
                    for n in range(len(result)):
                        loss[n]=mean_absolute_error(y_train[0],result[n])
                    loss_outlier=np.zeros(len(result2))
                    for n in range(len(result2)):
                        loss_outlier[n]=mean_absolute_error(y_train[0],result2[n])
                    error_record.append(loss_outlier)
                    #y_one=np.ones(len(loss))
                    #y_outlier=np.zeros(len(loss_outlier))
                    #data=np.concatenate([loss,loss_outlier])
                    #label=np.concatenate([y_one,y_outlier])
                    #plt.scatter(data,label)
                    #plt.hist([loss,loss_outlier],range=(0,0.3),stacked=False,bins=10)
                    m=0
                    AUC_score[i]=roc_auc_score(y_label,(-1)*loss_outlier)
                    for p in values:
                        y_pred=np.zeros(len(y_label))
                        threshold=int(len(loss)*p)+1
                        #print(loss[loss.argsort()[-threshold]])
                        y_pred[loss_outlier<=loss[loss.argsort()[-threshold]]]=1
                        #loss_outlier[loss_outlier>loss[loss.argsort()[-threshold]]
                        unseen_score[i][m]=balanced_accuracy_score(y_label,y_pred)
                        m+=1
                #print(unseen_score)
                #outputs.append(unseen_score)
                outputs2.append(AUC_score)
                outputs3.append(unseen_score)
                ensemble.append(error_record)
            outputs5.append(ensemble)
            outputs.append(outputs2)
            outputs4.append(outputs3)
    #makefile(outputs,"result_ocitn/cifar10_"+str(goal)+".pkl")

Train on 4146 samples, validate on 1243 samples
Epoch 1/100
4146/4146 [==============================] - 5s 1ms/step - loss: 32.4162 - val_loss: 21.4352
Epoch 2/100
4146/4146 [==============================] - 2s 559us/step - loss: 20.2573 - val_loss: 19.1542
Epoch 3/100
4146/4146 [==============================] - 2s 567us/step - loss: 18.4527 - val_loss: 17.8038
Epoch 4/100
4146/4146 [==============================] - 3s 675us/step - loss: 17.3119 - val_loss: 16.7839
Epoch 5/100
4146/4146 [==============================] - 3s 713us/step - loss: 16.3791 - val_loss: 15.9212
Epoch 6/100
4146/4146 [==============================] - 2s 518us/step - loss: 15.8090 - val_loss: 15.6635
Epoch 7/100
4146/4146 [==============================] - 2s 510us/step - loss: 15.1037 - val_loss: 15.2415
Epoch 8/100
4146/4146 [==============================] - 2s 507us/step - loss: 14.7641 - val_loss: 14.4504
Epoch 9/100
4146/4146 [==============================] - 2s 530us/step - loss: 14.3209 - val_loss:

Epoch 15/100
4170/4170 [==============================] - 2s 509us/step - loss: 15.0902 - val_loss: 14.6370
Epoch 16/100
4170/4170 [==============================] - 2s 511us/step - loss: 14.6920 - val_loss: 14.7522
Epoch 17/100
4170/4170 [==============================] - 2s 511us/step - loss: 14.6264 - val_loss: 14.2678
Epoch 18/100
4170/4170 [==============================] - 2s 510us/step - loss: 14.4738 - val_loss: 14.1289
Epoch 19/100
4170/4170 [==============================] - 2s 496us/step - loss: 14.3032 - val_loss: 13.9521
Epoch 20/100
4170/4170 [==============================] - 2s 497us/step - loss: 13.9758 - val_loss: 13.7887
Epoch 21/100
4170/4170 [==============================] - 2s 496us/step - loss: 13.8524 - val_loss: 13.7540
Epoch 22/100
4170/4170 [==============================] - 2s 497us/step - loss: 13.6820 - val_loss: 13.4582
Epoch 23/100
4170/4170 [==============================] - 2s 497us/step - loss: 13.6173 - val_loss: 13.3475
Epoch 24/100
4170/4170 [====

4291/4291 [==============================] - 2s 518us/step - loss: 11.9059 - val_loss: 11.9149
Epoch 46/100
4291/4291 [==============================] - 2s 513us/step - loss: 11.8450 - val_loss: 11.9033
Epoch 47/100
4291/4291 [==============================] - 2s 510us/step - loss: 11.7826 - val_loss: 12.5496
Epoch 48/100
4291/4291 [==============================] - 2s 514us/step - loss: 11.7760 - val_loss: 12.0017
Train on 4089 samples, validate on 1227 samples
Epoch 1/100
4089/4089 [==============================] - 2s 611us/step - loss: 34.1052 - val_loss: 22.8932
Epoch 2/100
4089/4089 [==============================] - 2s 504us/step - loss: 21.5082 - val_loss: 19.9775
Epoch 3/100
4089/4089 [==============================] - 2s 498us/step - loss: 19.1598 - val_loss: 18.2416
Epoch 4/100
4089/4089 [==============================] - 2s 504us/step - loss: 17.8580 - val_loss: 17.4585
Epoch 5/100
4089/4089 [==============================] - 2s 513us/step - loss: 16.8605 - val_loss: 16.365

3794/3794 [==============================] - 2s 502us/step - loss: 13.3995 - val_loss: 13.6788
Epoch 25/100
3794/3794 [==============================] - 2s 503us/step - loss: 13.1538 - val_loss: 13.1252
Epoch 26/100
3794/3794 [==============================] - 2s 505us/step - loss: 13.1786 - val_loss: 13.2257
Epoch 27/100
3794/3794 [==============================] - 2s 505us/step - loss: 13.0083 - val_loss: 12.9894
Epoch 28/100
3794/3794 [==============================] - 2s 502us/step - loss: 12.9185 - val_loss: 13.2864
Epoch 29/100
3794/3794 [==============================] - 2s 507us/step - loss: 12.9337 - val_loss: 13.3825
Epoch 30/100
3794/3794 [==============================] - 2s 509us/step - loss: 12.7370 - val_loss: 12.7788
Epoch 31/100
3794/3794 [==============================] - 2s 504us/step - loss: 12.6634 - val_loss: 12.7922
Epoch 32/100
3794/3794 [==============================] - 2s 522us/step - loss: 12.7113 - val_loss: 13.1435
Epoch 33/100
3794/3794 [=================

4142/4142 [==============================] - 2s 501us/step - loss: 10.4058 - val_loss: 10.2784
Epoch 48/100
4142/4142 [==============================] - 2s 499us/step - loss: 10.3587 - val_loss: 10.2869
Epoch 49/100
4142/4142 [==============================] - 2s 493us/step - loss: 10.3898 - val_loss: 10.4502
Epoch 50/100
4142/4142 [==============================] - 2s 500us/step - loss: 10.2894 - val_loss: 10.2391
Epoch 51/100
4142/4142 [==============================] - 2s 490us/step - loss: 10.3484 - val_loss: 10.3266
Epoch 52/100
4142/4142 [==============================] - 2s 511us/step - loss: 10.2099 - val_loss: 10.2974
Epoch 53/100
4142/4142 [==============================] - 2s 504us/step - loss: 10.1621 - val_loss: 10.2952
Epoch 54/100
4142/4142 [==============================] - 2s 509us/step - loss: 10.0923 - val_loss: 10.3424
Epoch 55/100
4142/4142 [==============================] - 2s 507us/step - loss: 10.5211 - val_loss: 10.1558
Epoch 56/100
4142/4142 [=================

...

4089/4089 [==============================] - 2s 505us/step - loss: 14.5319 - val_loss: 14.9212
Epoch 12/100
4089/4089 [==============================] - 2s 499us/step - loss: 14.2676 - val_loss: 13.9174
Epoch 13/100
4089/4089 [==============================] - 2s 499us/step - loss: 14.2344 - val_loss: 13.8364
Epoch 14/100
4089/4089 [==============================] - 2s 503us/step - loss: 13.8372 - val_loss: 13.8381
Epoch 15/100
4089/4089 [==============================] - 2s 501us/step - loss: 13.6200 - val_loss: 13.4141
Epoch 16/100
4089/4089 [==============================] - 2s 503us/step - loss: 13.5816 - val_loss: 13.3059
Epoch 17/100
4089/4089 [==============================] - 2s 512us/step - loss: 13.2581 - val_loss: 13.1523
Epoch 18/100
4089/4089 [==============================] - 2s 503us/step - loss: 13.0675 - val_loss: 13.3018
Epoch 19/100
4089/4089 [==============================] - 2s 502us/step - loss: 12.9725 - val_loss: 12.5355
Epoch 20/100
4089/4089 [=================

3794/3794 [==============================] - 2s 503us/step - loss: 15.1881 - val_loss: 16.6911
Epoch 14/100
3794/3794 [==============================] - 2s 507us/step - loss: 14.9211 - val_loss: 14.8823
Epoch 15/100
3794/3794 [==============================] - 2s 509us/step - loss: 14.6919 - val_loss: 14.7462
Epoch 16/100
3794/3794 [==============================] - 2s 506us/step - loss: 14.5961 - val_loss: 14.4361
Epoch 17/100
3794/3794 [==============================] - 2s 548us/step - loss: 14.5814 - val_loss: 15.8732
Epoch 18/100
3794/3794 [==============================] - 2s 516us/step - loss: 14.3286 - val_loss: 14.2020
Epoch 19/100
3794/3794 [==============================] - 2s 511us/step - loss: 14.2068 - val_loss: 14.3918
Epoch 20/100
3794/3794 [==============================] - 2s 509us/step - loss: 14.0138 - val_loss: 14.0953
Epoch 21/100
3794/3794 [==============================] - 2s 507us/step - loss: 13.9341 - val_loss: 14.4191
Epoch 22/100
3794/3794 [=================

4142/4142 [==============================] - 2s 506us/step - loss: 11.0017 - val_loss: 11.1801
Epoch 41/100
4142/4142 [==============================] - 2s 507us/step - loss: 11.0088 - val_loss: 11.0691
Epoch 42/100
4142/4142 [==============================] - 2s 506us/step - loss: 11.1070 - val_loss: 10.9191
Epoch 43/100
4142/4142 [==============================] - 2s 504us/step - loss: 10.9333 - val_loss: 10.8809
Epoch 44/100
4142/4142 [==============================] - 2s 506us/step - loss: 10.9760 - val_loss: 11.1909
Epoch 45/100
4142/4142 [==============================] - 2s 512us/step - loss: 10.8500 - val_loss: 11.0841
Epoch 46/100
4142/4142 [==============================] - 2s 503us/step - loss: 10.8657 - val_loss: 10.8146
Epoch 47/100
4142/4142 [==============================] - 2s 506us/step - loss: 11.0214 - val_loss: 10.8214
Epoch 48/100
4142/4142 [==============================] - 2s 502us/step - loss: 10.7002 - val_loss: 10.7235
Epoch 49/100
4142/4142 [=================

4095/4095 [==============================] - 2s 512us/step - loss: 13.1116 - val_loss: 13.1146
Epoch 25/100
4095/4095 [==============================] - 2s 507us/step - loss: 13.1373 - val_loss: 12.9173
Epoch 26/100
4095/4095 [==============================] - 2s 515us/step - loss: 12.9405 - val_loss: 12.8869
Epoch 27/100
4095/4095 [==============================] - 2s 510us/step - loss: 12.9787 - val_loss: 13.3314
Epoch 28/100
4095/4095 [==============================] - 2s 511us/step - loss: 12.8064 - val_loss: 12.5846
Epoch 29/100
4095/4095 [==============================] - 2s 516us/step - loss: 12.7291 - val_loss: 13.0806
Epoch 30/100
4095/4095 [==============================] - 2s 514us/step - loss: 12.6746 - val_loss: 12.7552
Epoch 31/100
4095/4095 [==============================] - 2s 516us/step - loss: 12.6271 - val_loss: 12.5571
Epoch 32/100
4095/4095 [==============================] - 2s 509us/step - loss: 12.4571 - val_loss: 12.5909
Epoch 33/100
4095/4095 [=================

4164/4164 [==============================] - 2s 503us/step - loss: 10.5867 - val_loss: 10.5274
Epoch 47/100
4164/4164 [==============================] - 2s 511us/step - loss: 10.5781 - val_loss: 10.4658
Epoch 48/100
4164/4164 [==============================] - 2s 510us/step - loss: 10.5604 - val_loss: 10.6884
Epoch 49/100
4164/4164 [==============================] - 2s 512us/step - loss: 10.4896 - val_loss: 10.4047
Epoch 50/100
4164/4164 [==============================] - 2s 507us/step - loss: 10.4187 - val_loss: 10.3234
Epoch 51/100
4164/4164 [==============================] - 2s 508us/step - loss: 10.4222 - val_loss: 10.2880
Epoch 52/100
4164/4164 [==============================] - 2s 514us/step - loss: 10.4313 - val_loss: 10.8084
Epoch 53/100
4164/4164 [==============================] - 2s 510us/step - loss: 10.3827 - val_loss: 10.3275
Epoch 54/100
4164/4164 [==============================] - 2s 510us/step - loss: 10.3745 - val_loss: 10.5602
Epoch 55/100
4164/4164 [=================

In [13]:
outputs6=[]
for ensemble in outputs5:
    CEtest2=None
    value = 0
    for scores in ensemble:
        if type(CEtest2)==type(None):
            CEtest2=np.array(scores)
        else:
            print("OK")
            CEtest2=CEtest2+np.array(scores)
    AUC_ensemble=np.zeros(10)
    for i in range(10):
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        AUC_ensemble[i]=roc_auc_score(y_label,(-1)*CEtest2[i])
    outputs6.append(AUC_ensemble)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [14]:
np.array(outputs6).mean(axis=0)

array([0.99545982, 0.99810982, 0.98034778, 0.9647425 , 0.96291041,
       0.98058199, 0.99606404, 0.96544179, 0.97860115, 0.98274363])

In [15]:
np.array(outputs6).std(axis=0)

array([3.91609680e-04, 5.08096833e-05, 9.95095951e-04, 2.54968066e-03,
       1.62219797e-03, 2.12687732e-03, 1.08891375e-04, 2.91576028e-03,
       1.30834173e-03, 3.52158863e-04])

In [16]:
AUC_score2=[]
for ensemble in outputs5:
    AUC_score=[]
    for scores in ensemble:
        AUC=np.zeros(10)
        for i in range(10):
            y_label=np.zeros(len(y_test))
            y_label[y_test==i]=1
            AUC[i]=roc_auc_score(y_label,(-1)*scores[i])
        AUC_score.append(AUC)
    AUC_score2.append(AUC_score)

In [17]:
np.array(AUC_score2).mean(axis=0)

array([[0.99519781, 0.99746011, 0.9710794 , 0.94493932, 0.95499507,
        0.97790843, 0.99466884, 0.94477159, 0.97007476, 0.97385314],
       [0.99058582, 0.99786916, 0.98134342, 0.95828334, 0.96558141,
        0.97741287, 0.99283237, 0.96283739, 0.96695747, 0.98235844],
       [0.99442518, 0.99747385, 0.96200929, 0.96385473, 0.94846783,
        0.9675624 , 0.99418409, 0.96771267, 0.97397825, 0.97995416],
       [0.99363351, 0.99741022, 0.9766168 , 0.94956245, 0.96649114,
        0.96974961, 0.99568065, 0.96082603, 0.97483349, 0.97851878]])

In [18]:
np.array(AUC_score2).std(axis=0)

array([[0.00035749, 0.00023809, 0.00347481, 0.00482067, 0.00347132,
        0.00462293, 0.00039312, 0.00572398, 0.00229066, 0.00078122],
       [0.00125687, 0.0003222 , 0.0031753 , 0.00363918, 0.00260157,
        0.00395146, 0.00046022, 0.00540032, 0.00234657, 0.00129085],
       [0.00070341, 0.00015356, 0.00369938, 0.00438485, 0.00495039,
        0.00374487, 0.00054002, 0.00286936, 0.00290725, 0.00198762],
       [0.00128513, 0.00038893, 0.00152995, 0.00713351, 0.00396014,
        0.0029169 , 0.00034048, 0.00638817, 0.00292044, 0.00090357]])

In [24]:
np.array(outputs6).mean(axis=0).mean()

0.9805002934514713

In [20]:
np.array(outputs6).std(axis=0)

array([3.91609680e-04, 5.08096833e-05, 9.95095951e-04, 2.54968066e-03,
       1.62219797e-03, 2.12687732e-03, 1.08891375e-04, 2.91576028e-03,
       1.30834173e-03, 3.52158863e-04])

In [21]:
outputs7=[]
outputs7.append(np.array(AUC_score2).mean(axis=0))
outputs7.append(np.array(AUC_score2).std(axis=0))
outputs7.append(np.array(outputs6).mean(axis=0))
outputs7.append(np.array(outputs6).std(axis=0))

In [22]:
outputs7

[array([[0.99519781, 0.99746011, 0.9710794 , 0.94493932, 0.95499507,
         0.97790843, 0.99466884, 0.94477159, 0.97007476, 0.97385314],
        [0.99058582, 0.99786916, 0.98134342, 0.95828334, 0.96558141,
         0.97741287, 0.99283237, 0.96283739, 0.96695747, 0.98235844],
        [0.99442518, 0.99747385, 0.96200929, 0.96385473, 0.94846783,
         0.9675624 , 0.99418409, 0.96771267, 0.97397825, 0.97995416],
        [0.99363351, 0.99741022, 0.9766168 , 0.94956245, 0.96649114,
         0.96974961, 0.99568065, 0.96082603, 0.97483349, 0.97851878]]),
 array([[0.00035749, 0.00023809, 0.00347481, 0.00482067, 0.00347132,
         0.00462293, 0.00039312, 0.00572398, 0.00229066, 0.00078122],
        [0.00125687, 0.0003222 , 0.0031753 , 0.00363918, 0.00260157,
         0.00395146, 0.00046022, 0.00540032, 0.00234657, 0.00129085],
        [0.00070341, 0.00015356, 0.00369938, 0.00438485, 0.00495039,
         0.00374487, 0.00054002, 0.00286936, 0.00290725, 0.00198762],
        [0.00128513, 0.00

In [23]:
makefile(outputs7,"ITNensemble/mnist.pkl")